# Аналіз тексту книги 'Alice's Adventures in Wonderland'

## Опис завдання
1. Завантаження тексту з веб-сторінки.
2. Попередня обробка тексту (очищення, видалення стоп-слів, перетворення до нижнього регістру).
3. Розбиття тексту на глави та аналіз кожної глави за допомогою TF-IDF.
4. Тематичне моделювання з використанням LDA.
5. Порівняння результатів та висновки.

In [1]:
import requests
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import string
from collections import Counter
import matplotlib.pyplot as plt

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Настя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Крок 1: Завантаження тексту книги

In [2]:
url = 'http://www.gutenberg.org/files/11/11-0.txt'
response = requests.get(url)
response.encoding = 'utf-8'
text = response.text

print(text[:1000])

*** START OF THE PROJECT GUTENBERG EBOOK 11 ***
[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-Hole
 CHAPTER II.    The Pool of Tears
 CHAPTER III.   A Caucus-Race and a Long Tale
 CHAPTER IV.    The Rabbit Sends in a Little Bill
 CHAPTER V.     Advice from a Caterpillar
 CHAPTER VI.    Pig and Pepper
 CHAPTER VII.   A Mad Tea-Party
 CHAPTER VIII.  The Queen’s Croquet-Ground
 CHAPTER IX.    The Mock Turtle’s Story
 CHAPTER X.     The Lobster Quadrille
 CHAPTER XI.    Who Stole the Tarts?
 CHAPTER XII.   Alice’s Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversat


## Крок 2: Попередня обробка тексту

In [3]:
def preprocess_text(text):
    # Приведення до нижнього регістру
    text = text.lower()
    # Видалення неалфавітних символів та цифр
    text = re.sub(r'[^a-z\s]', '', text)
    # Розбиття на слова та видалення стоп-слів
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

processed_text = preprocess_text(text)

print(processed_text[:500])

start project gutenberg ebook illustration alices adventures wonderland lewis carroll millennium fulcrum edition contents chapter rabbithole chapter ii pool tears chapter iii caucusrace long tale chapter iv rabbit sends little bill chapter v advice caterpillar chapter vi pig pepper chapter vii mad teaparty chapter viii queens croquetground chapter ix mock turtles story chapter x lobster quadrille chapter xi stole tarts chapter xii alices evidence chapter rabbithole alice beginning get tired sitt


## Крок 3: Розбиття тексту на глави

In [4]:
chapters = processed_text.split('chapter ')[1:]
chapters = [chapter.strip() for chapter in chapters]

print(f'Кількість глав: {len(chapters)}')
print(chapters[0][:500])  # Перевірка першої глави

Кількість глав: 24
rabbithole


## Крок 4: Розрахунок TF-IDF для кожної глави та відбір Топ-20 слів

In [5]:
vectorizer = TfidfVectorizer(max_features=20)
X = vectorizer.fit_transform(chapters)

top_words_per_chapter = {}
feature_names = vectorizer.get_feature_names_out()
for idx, chapter in enumerate(chapters):
    tfidf_scores = dict(zip(feature_names, X[idx].toarray().flatten()))
    sorted_words = sorted(tfidf_scores.items(), key=lambda item: item[1], reverse=True)[:20]
    top_words_per_chapter[f'Chapter {idx+1}'] = sorted_words

top_words_per_chapter

{'Chapter 1': [('alice', 0.0),
  ('began', 0.0),
  ('could', 0.0),
  ('dont', 0.0),
  ('im', 0.0),
  ('king', 0.0),
  ('know', 0.0),
  ('like', 0.0),
  ('little', 0.0),
  ('mock', 0.0),
  ('one', 0.0),
  ('queen', 0.0),
  ('said', 0.0),
  ('see', 0.0),
  ('thought', 0.0),
  ('time', 0.0),
  ('turtle', 0.0),
  ('well', 0.0),
  ('went', 0.0),
  ('would', 0.0)],
 'Chapter 2': [('alice', 0.0),
  ('began', 0.0),
  ('could', 0.0),
  ('dont', 0.0),
  ('im', 0.0),
  ('king', 0.0),
  ('know', 0.0),
  ('like', 0.0),
  ('little', 0.0),
  ('mock', 0.0),
  ('one', 0.0),
  ('queen', 0.0),
  ('said', 0.0),
  ('see', 0.0),
  ('thought', 0.0),
  ('time', 0.0),
  ('turtle', 0.0),
  ('well', 0.0),
  ('went', 0.0),
  ('would', 0.0)],
 'Chapter 3': [('alice', 0.0),
  ('began', 0.0),
  ('could', 0.0),
  ('dont', 0.0),
  ('im', 0.0),
  ('king', 0.0),
  ('know', 0.0),
  ('like', 0.0),
  ('little', 0.0),
  ('mock', 0.0),
  ('one', 0.0),
  ('queen', 0.0),
  ('said', 0.0),
  ('see', 0.0),
  ('thought', 0.0),
  (

## Крок 5: Тематичне моделювання з використанням LDA

In [6]:
texts = [chapter.split() for chapter in chapters]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda_model = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=15)

lda_topics = lda_model.print_topics(num_words=10)
for topic in lda_topics:
    print(topic)

(0, '0.030*"said" + 0.029*"alice" + 0.012*"little" + 0.007*"like" + 0.007*"im" + 0.007*"went" + 0.007*"one" + 0.007*"thought" + 0.006*"duchess" + 0.006*"know"')
(1, '0.025*"alice" + 0.019*"said" + 0.012*"queen" + 0.008*"like" + 0.007*"little" + 0.007*"one" + 0.007*"see" + 0.006*"went" + 0.006*"thought" + 0.006*"think"')
(2, '0.040*"said" + 0.029*"alice" + 0.009*"know" + 0.009*"hatter" + 0.009*"gryphon" + 0.008*"mock" + 0.008*"turtle" + 0.007*"dormouse" + 0.007*"would" + 0.007*"time"')
(3, '0.037*"said" + 0.020*"king" + 0.016*"alice" + 0.009*"hatter" + 0.008*"one" + 0.007*"queen" + 0.007*"court" + 0.007*"would" + 0.007*"rabbit" + 0.007*"little"')
(4, '0.002*"caterpillar" + 0.002*"pool" + 0.002*"tears" + 0.002*"ii" + 0.002*"v" + 0.002*"advice" + 0.001*"said" + 0.001*"alice" + 0.000*"one" + 0.000*"little"')


## Крок 6: Порівняння результатів TF-IDF та LDA

In [7]:
print("TF-IDF допомагає виявити важливі слова в окремих главах, тоді як LDA допомагає знайти теми в усьому тексті.")

TF-IDF допомагає виявити важливі слова в окремих главах, тоді як LDA допомагає знайти теми в усьому тексті.
